### Step 0: Install & import libraries

In [1]:
# !pip install beautifulsoup4
# !pip install requests
# !pip install tqdm

In [2]:
import bs4 as bs
import requests
from lxml import etree

from tqdm import tqdm

import pandas as pd

from unidecode import unidecode

### Step 1: Web scrape regions & their capitals
We found [this page](https://www.sport-histoire.fr/es/Geografia/Lista_departamentos_regiones_Peru.php) to get regions and their respective capitals

In [3]:
resp = requests.get('https://www.sport-histoire.fr/es/Geografia/Lista_departamentos_regiones_Peru.php') # "get" request to read data without making a change
resp = resp.text                       # fetch the HTML data
soup = bs.BeautifulSoup(resp, 'lxml')  # convert to BeautifulSoup-type object to work with HTML efficiently
                                       # lxml allows easy handling of HTML files/objects
table = soup.find('table', {'class': 'table-responsive-sm'})  # detect table by using 'table-responsive-sm' class

In [4]:
regions = []
capitals = []
        
for row in tqdm(table.findAll('tr')[1:]):
    if len(row.findAll('td')) == 4:
        region = row.findAll('td')[1].text
        capital = row.findAll('td')[2].text
        regions.append(region)
        capitals.append(capital)
        
    else: # this is for "Lima (departamento)" case
        region = row.findAll('td')[0].text
        capital = row.findAll('td')[1].text
        regions.append(region)
        capitals.append(capital)

100%|███████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 26787.50it/s]


In [5]:
df = pd.DataFrame(list(zip(regions, capitals)), columns =['Regions', 'Capitals'])
df

,Regions,Capitals
0,Amazonas,Chachapoyas
1,Áncash,Huaraz
2,Apurímac,Abancay
3,Arequipa,Arequipa
4,Ayacucho,Ayacucho
5,Cajamarca,Cajamarca
6,Callao,Callao
7,Cusco,Cuzco
8,Huancavelica,Huancavelica
9,Huánuco,Huánuco


### Step 2: Transform capitals
We need to convert capitals to certain format in order to search for their geographic coordinates at [geodatos](https://www.geodatos.net/coordenadas/peru) page
<br>
<br><b>For example:</b>
- Chachapoyas -> [chachapoyas](https://www.geodatos.net/coordenadas/peru/chachapoyas)
- Puerto Maldonado -> [puerto-maldonado](https://www.geodatos.net/coordenadas/peru/puerto-maldonado)
- Cerro de Pasco -> [cerro-de-pasco](https://www.geodatos.net/coordenadas/peru/cerro-de-pasco)

In [6]:
def transform_capital(x):
    x = x.lower()           # convert to lowercase
    x = x.replace(' ','-')  # replace blank space to "-"
    x = unidecode(x)        # remove accents
    return x

In [7]:
df["modifiedCapitals"] = df["Capitals"].apply(lambda x: transform_capital(x))
df["modifiedCapitals"] = df["modifiedCapitals"].replace('cuzco','cusco') # cuzco does not exist in geodatos page
df

,Regions,Capitals,modifiedCapitals
0,Amazonas,Chachapoyas,chachapoyas
1,Áncash,Huaraz,huaraz
2,Apurímac,Abancay,abancay
3,Arequipa,Arequipa,arequipa
4,Ayacucho,Ayacucho,ayacucho
5,Cajamarca,Cajamarca,cajamarca
6,Callao,Callao,callao
7,Cusco,Cuzco,cusco
8,Huancavelica,Huancavelica,huancavelica
9,Huánuco,Huánuco,huanuco


### Step 3: Web scrape capitals geographic coordinates (latitude & longitude)
We use [this page](https://www.geodatos.net/coordenadas/peru/) to get geographic coordinates (decimal coordinates) of peruvian regions
<br>
<img src="Resources/decimal_coordinates.jpg" width="700px">  

In [8]:
df['Latitude'] = ""
df['Longitude'] = ""

for i in tqdm(range(len(df))):
    capital = df['modifiedCapitals'][i]
    webpage = requests.get('https://www.geodatos.net/coordenadas/peru/' + capital)
    soup = bs.BeautifulSoup(webpage.content, "html.parser")
    coordinates = etree.HTML(str(soup))
    coordinates = coordinates.xpath('/html/body/div[2]/div[1]/div[1]/p')[0].text # -6.23169, -77.86903
    
    lat = float(coordinates.split(', ')[0]) # -6.23169
    lng = float(coordinates.split(', ')[1]) # -77.86903
    
    df.at[i,'Latitude'] = lat
    df.at[i,'Longitude'] = lng

df

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:08<00:00,  2.93it/s]


,Regions,Capitals,modifiedCapitals,Latitude,Longitude
0,Amazonas,Chachapoyas,chachapoyas,-6.23169,-77.86903
1,Áncash,Huaraz,huaraz,-9.52779,-77.52778
2,Apurímac,Abancay,abancay,-13.63389,-72.88139
3,Arequipa,Arequipa,arequipa,-16.39889,-71.535
4,Ayacucho,Ayacucho,ayacucho,-13.15878,-74.22321
5,Cajamarca,Cajamarca,cajamarca,-7.16378,-78.50027
6,Callao,Callao,callao,-12.05659,-77.11814
7,Cusco,Cuzco,cusco,-13.52264,-71.96734
8,Huancavelica,Huancavelica,huancavelica,-12.78261,-74.97266
9,Huánuco,Huánuco,huanuco,-9.93062,-76.24223


### Step 4: Get altitude of capitals by merging tables
We can get altitude (m.a.s.l.) info from this [INEI page](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1253/cap01/cap01012.xls) (it's been added to *Input* folder), then we manually manipulate this data to obtain "*Input/altitude_provinces.xlsx*" file that contains latitude of all provinces including each capital of Peruvian regions (in **bold**)
<br>
<img src="Resources/altitude_provinces.jpg" width="300px"> 

In [9]:
altitude = pd.read_excel('Input/altitude_provinces.xlsx')
altitude

,Name,Altitude
0,Chachapoyas,2339
1,Bagua,421
2,Jumbilla,1991
3,Santa María de Nieva,222
4,Lamud,2307
...,...,...
190,Zarumilla,14
191,Pucallpa,157
192,Atalaya,228
193,Aguaytía,300


Thus, this table (**altitude**) and our dataframe (**df**) will be merged by "*modifiedCapitals*" column to get a final data frame (**final_df**) containing latitude, longitude and altitude information.

In [10]:
altitude["modifiedName"] = altitude["Name"].apply(lambda x: transform_capital(x))
final_df = df.merge(altitude[['modifiedName','Altitude']].rename(columns={'modifiedName':'modifiedCapitals'}), 
                    on='modifiedCapitals',how='left')
final_df = final_df.drop(columns=['modifiedCapitals'])
final_df

,Regions,Capitals,Latitude,Longitude,Altitude
0,Amazonas,Chachapoyas,-6.23169,-77.86903,2339
1,Áncash,Huaraz,-9.52779,-77.52778,3038
2,Apurímac,Abancay,-13.63389,-72.88139,2392
3,Arequipa,Arequipa,-16.39889,-71.535,2337
4,Ayacucho,Ayacucho,-13.15878,-74.22321,2760
5,Cajamarca,Cajamarca,-7.16378,-78.50027,2719
6,Callao,Callao,-12.05659,-77.11814,3
7,Cusco,Cuzco,-13.52264,-71.96734,3414
8,Huancavelica,Huancavelica,-12.78261,-74.97266,3679
9,Huánuco,Huánuco,-9.93062,-76.24223,1898


In [11]:
df.to_csv("Output/Coordinates.csv", index=False, encoding='utf-8')
df.to_excel("Output/Coordinates.xlsx", index=False, encoding='utf-8')